# UCS - Uniform Cost Search (Priority queue)

In [6]:
class Graph:
    def __init__(self, filename):
        self.nodes = []  
        self.edges = {}  
        self.load_from_file(filename)

    def load_from_file(self, filename):
        with open(filename, "r") as file:
            lines = file.readlines()

        first_line = lines[0].strip().split()
        n, m = int(first_line[0]), int(first_line[1])

        self.nodes = [Node(str(i)) for i in range(n)]
        self.edges = {node.name: {} for node in self.nodes}

        for line in lines[1:]:
            u, v, *w = map(int, line.strip().split())
            weight = w[0] if w else 1
            self.edges[str(u)][str(v)] = weight
            self.edges[str(v)][str(u)] = weight  

class Node:
    def __init__(self, name):
        self.name = name

 **Lista de adyacencia**

In [7]:
class ListaAdyacencia:
    def __init__(self, graph):
        self.adj_list = {}
        self.build_from_graph(graph)

    def build_from_graph(self, graph):
        for node in graph.nodes:
            self.adj_list[node.name] = list(graph.edges[node.name].keys())

    def display(self):
        print("Lista de Adyacencia:")
        for node, neighbors in self.adj_list.items():
            print(f"{node}: {neighbors}")

In [8]:
graph = Graph("grafo_ucs.txt")

# Lista de adyacencia
Lista_adyacencia = ListaAdyacencia(graph)
Lista_adyacencia.display()

Lista de Adyacencia:
0: ['1', '2']
1: ['0', '2', '3']
2: ['0', '1', '3']
3: ['1', '2']


**Lista de adyacencia ponderada**

In [9]:
class ListaAdyacenciaPonderada:
    def __init__(self, graph):
        self.weighted_adj_list = {}
        self.build_from_graph(graph)

    def build_from_graph(self, graph):
        for node in graph.nodes:
            self.weighted_adj_list[node.name] = [(neighbor, weight) for neighbor, weight in graph.edges[node.name].items()]

    def display(self):
        print("Lista de Adyacencia Ponderada:")
        for node, neighbors in self.weighted_adj_list.items():
            print(f"{node}: {neighbors}")

In [10]:
graph = Graph("grafo_ucs.txt")

Adyacencia_podnderada = ListaAdyacenciaPonderada(graph)
Adyacencia_podnderada.display()

Lista de Adyacencia Ponderada:
0: [('1', 2), ('2', 4)]
1: [('0', 2), ('2', 1), ('3', 7)]
2: [('0', 4), ('1', 1), ('3', 3)]
3: [('1', 7), ('2', 3)]


**Matriz binaria**

In [28]:
class MatrizBinaria:
    def __init__(self, graph):
        self.matrix = []
        self.build_from_graph(graph)

    def build_from_graph(self, graph):
        n = len(graph.nodes)
        node_indices = {node.name: idx for idx, node in enumerate(graph.nodes)}
        self.matrix = [[0] * n for _ in range(n)]

        for node in graph.nodes:
            u = node_indices[node.name]
            for neighbor in graph.edges[node.name]:
                v = node_indices[neighbor]
                self.matrix[u][v] = 1

    def display(self, nodes):
        print("Matriz Binaria:")
        for row in self.matrix:
            print(row)

In [29]:
graph = Graph("grafo_ucs.txt")

Matriz_binaria = MatrizBinaria(graph)
Matriz_binaria.display(graph.nodes)

Matriz Binaria:
[0, 1, 1, 0]
[1, 0, 1, 1]
[1, 1, 0, 1]
[0, 1, 1, 0]


**Matriz de Distancias**

In [13]:
class DistanceMatrix:
    def __init__(self, graph):
        self.matrix = []
        self.build_from_graph(graph)

    def build_from_graph(self, graph):
        n = len(graph.nodes)
        node_indices = {node.name: idx for idx, node in enumerate(graph.nodes)}
        self.matrix = [[float('inf')] * n for _ in range(n)]

        for i in range(n):
            self.matrix[i][i] = 0

        for node in graph.nodes:
            u = node_indices[node.name]
            for neighbor, weight in graph.edges[node.name].items():
                v = node_indices[neighbor]
                self.matrix[u][v] = weight

    def display(self, nodes):
        print("Matriz de Distancias:")
        for row in self.matrix:
            print(row)

In [14]:
graph = Graph("grafo_ucs.txt")

distance_matrix = DistanceMatrix(graph)
distance_matrix.display(graph.nodes)

Matriz de Distancias:
[0, 2, 4, inf]
[2, 0, 1, 7]
[4, 1, 0, 3]
[inf, 7, 3, 0]


**Algoritmo UCS**

In [41]:
class UCS:
    def __init__(self, graph):
        self.graph = graph

    def search(self, start, goal):
        from heapq import heappop, heappush

        frontier = [(0, start)] 
        expanded = set()       
        came_from = {start: None} 
        cost_so_far = {start: 0}  

        while frontier:
            # Extraer el nodo con el menor costo acumulado
            current_cost, current_node = heappop(frontier)

            # Si el nodo actual ya fue expandido, lo saltamos
            if current_node in expanded:
                continue

            # Si encontramos el nodo objetivo, reconstruimos el camino
            if current_node == goal:
                path = []
                while current_node is not None:
                    path.append(current_node)
                    current_node = came_from[current_node]
                return path[::-1], current_cost

            # Marcar el nodo como expandido
            expanded.add(current_node)

            # Explorar los vecinos del nodo actual
            for neighbor, weight in self.graph.edges[current_node].items():
                new_cost = current_cost + weight

                # Si el vecino no está expandido ni en la frontera, o si encontramos un menor costo
                if neighbor not in expanded and (neighbor not in cost_so_far or new_cost < cost_so_far[neighbor]):
                    cost_so_far[neighbor] = new_cost
                    came_from[neighbor] = current_node
                    heappush(frontier, (new_cost, neighbor))

        return None, float('inf')

In [48]:
# Crear el grafo desde el archivo
graph = Graph("grafo_ucs.txt")

# Crear una instancia del algoritmo UCS
ucs_solver = UCS(graph)

# Definir el nodo inicial y el nodo objetivo
start_node = "2" 
goal_node = "2"   

# Ejecutar la búsqueda UCS
path, cost = ucs_solver.search(start_node, goal_node)

print(f"Camino encontrado (UCS): {path} con costo {cost}")

Camino encontrado (UCS): ['2'] con costo 0
